In [3]:
import numpy as np

In [ ]:
rng = np.random.default_rng(1212)

n = 20
D = np.linspace(1, n, num=n)
X = rng.random((n, n))
A = X @ np.diag(D) @ np.linalg.inv(X)
u, v = rng.random((2, n))

In [92]:
@np.vectorize
def f(z):
    return u.T @ np.linalg.solve(A - z * np.eye(n), v)


@np.vectorize
def f_z(z):
    return -u.T @ np.linalg.solve(
        A - z * np.eye(n), np.linalg.solve(A - z * np.eye(n), v)
    )